# 오산시 어린이 교통사고 위험지역 도출 

## 1. Data 불러오기

In [ ]:
#12/24 GeoSeries 추가
from geoband.API import *
import pandas as pd
import folium
import json
import geopandas as gpd
from geojson import Feature, FeatureCollection, Point, dump
from geopandas import GeoSeries
from shapely.geometry import Point
import matplotlib.pyplot as plt
import pydeck as pdk
import shapely
import numpy as np
plt.rc("font", family = "Malgun Gothic")
%matplotlib inline
import warnings
warnings.filterwarnings(action='ignore') 
from sklearn.linear_model import LinearRegression
from sklearn import linear_model

In [ ]:
path = '/opt/app-root/src/Osan_si/Osan_data_preprocessing/'
df_osan_grid=gpd.read_file(path + 'df_osan_grid.geojson')
df_road_info=gpd.read_file(path + 'df_road_info.geojson')

df1_parking=gpd.read_file(path + 'df1_parking.geojson')
df2_grid_accident=gpd.read_file(path + 'df2_grid_accident.geojson')
df4_pop=gpd.read_file(path + 'df4_pop.geojson')
df5_junior_pop=gpd.read_file(path + 'df5_junior_pop.geojson')
df6_product_pop=gpd.read_file(path + 'df6_product_pop.geojson')
df7_senior_pop=gpd.read_file(path + 'df7_senior_pop.geojson')
df8_floating_pop=gpd.read_file(path + 'df8_floating_pop.geojson')
df9_protection_area=gpd.read_file(path + 'df9_protection_area.geojson')
df10_school=gpd.read_file(path + 'df10_school.geojson')
df11_elementary_district=gpd.read_file(path + 'df11_elementary_district.geojson')
df12_middle_district=gpd.read_file(path + 'df12_middle_district.geojson')
df13_kinder=gpd.read_file(path + 'df13_kinder.geojson')
df15_traffic_cctv=gpd.read_file(path + 'df15_traffic_cctv.geojson')
df16_road_sign=gpd.read_file(path + 'df16_road_sign.geojson')
df17_crosswalk=gpd.read_file(path + 'df17_crosswalk.geojson')
df18_speed_bump=gpd.read_file(path + 'df18_speed_bump.geojson')
df19_traffic_light=gpd.read_file(path + 'df19_traffic_light.geojson')
df20_cctv=gpd.read_file(path + 'df20_cctv.geojson')
df21_sidewalk=gpd.read_file(path + 'df21_sidewalk.geojson')
df22_bus_stop=gpd.read_file(path + 'df22_bus_stop.geojson')
df23_road=gpd.read_file(path + 'df23_road.geojson')
df24_traffic=pd.read_csv(path + 'df24_traffic.csv')
df25_traffic_frequency=pd.read_csv(path + 'df25_traffic_frequency.csv')
df26_traffic_time=pd.read_csv(path + 'df26_traffic_time.csv')
df27_building=gpd.read_file(path + 'df27_building.geojson')
df28_grid_building=gpd.read_file(path + 'df28_grid_building.geojson')
df29_sports=gpd.read_file(path + 'df29_sports.geojson')
df30_academy=gpd.read_file(path + 'df30_academy.geojson')

## 2. 분석 기초 함수

### 1) pydeck 사용을 위해 geometry의 Point, Multilinestring, Multipolygon을 coordinates로 바꿔주기 위한 함수 선언

> 점에서 좌표로

In [ ]:
# 12/25 창균 추가
def POINT_to_coordinates(geo_data):
    geo_data['lat'] = geo_data['geometry'].apply(lambda coord: coord.y)
    geo_data['lon'] = geo_data['geometry'].apply(lambda coord: coord.x)
    return geo_data

# 사용 예
# df19_traffic_light = POINT_to_coordinates(df19_traffic_light)

> 라인에서 좌표로

In [ ]:
# 12/25 창균 추가
def MULTILINESTRING_to_coordinates(line_string):
    if isinstance(line_string, shapely.geometry.linestring.LineString):
        lon, lat = line_string.xy
        return [[x, y] for x, y in zip(lon, lat)]
    elif isinstance(line_string, shapely.geometry.multilinestring.MultiLineString):
        ret = []
        for i in range(len(line_string)):
            lon, lat = line_string[i].xy
            for x, y in zip(lon, lat):
                ret.append([x, y])
        return ret
    
# 사용 예
# df23_road['coordinates'] = df23_road['geometry'].apply(MULTILINESTRING_to_coordinates)
# df23_road = pd.DataFrame(df23_road) # geopanadas 가 아닌 pandas 의 데이터프레임으로 꼭 바꿔줘야 합니다.

> 면에서 좌표로

In [ ]:
# 12/25 창균 추가
def MULTIPOLYGON_to_coordinates(x):
    lon, lat = x[0].exterior.xy
    return [[x, y] for x, y in zip(lon, lat)]

# 사용 예
# df_osan_grid['coordinates'] = df_osan_grid['geometry'].apply(MULTIPOLYGON_to_coordinates)
# df_osan_grid = pd.DataFrame(df_osan_grid) # geopanadas 가 아닌 pandas 의 데이터프레임으로 꼭 바꿔줘야 합니다.

### 2) 각각의 df의 lon, lat을 geometry의 point로 변환

> 좌표에서 점으로

In [ ]:
# 12/24 정민 수정사항
def location_to_point(df):
    point_df = gpd.points_from_xy(df.lon, df.lat)
    point_df = GeoSeries(point_df)
    
    col = df.columns
    
    loc_df = gpd.GeoDataFrame(df[col], geometry = point_df)
    
    return loc_df

### 3) sjoin : 격자내 존재하는 좌표 개수의 합을 구하는 함수

In [ ]:
def sjoin(grid_info,df, new_col_name, option = 'contains'):
    joined= gpd.sjoin(grid_info,df, op=option)
    result = joined.groupby('gid').size()
    result = result.to_frame().reset_index()
    grid_info=pd.merge(grid_info,result, how='outer',on='gid')
    
    grid_info = grid_info.rename(columns = {0:new_col_name})
    grid_info=grid_info.fillna(0)
    
    return grid_info

## 3. EDA 및 시각화

> 1. pydeck 사용을 위한 추가처리

In [ ]:
# 다음 파일에서 실행할 애들
# 전처리과정에서 처리시 GeoJson으로 저장할 수 없음.
# 시각화할때 필요한 data들이라 시각화 전에만 입력해줘도 됨
# geometry to coordinates
df4_pop['coordinates'] = df4_pop['geometry'].apply(MULTIPOLYGON_to_coordinates)
# df4_pop = pd.DataFrame(df4_pop)

# geometry to coordinates
df5_junior_pop['coordinates'] = df5_junior_pop['geometry'].apply(MULTIPOLYGON_to_coordinates)
# df5_junior_pop = pd.DataFrame(df5_junior_pop)

# geometry to coordinates
df6_product_pop['coordinates'] = df6_product_pop['geometry'].apply(MULTIPOLYGON_to_coordinates)
# df6_product_pop = pd.DataFrame(df6_product_pop)

# geometry to coordinates
df7_senior_pop['coordinates'] = df7_senior_pop['geometry'].apply(MULTIPOLYGON_to_coordinates)
# df7_senior_pop = pd.DataFrame(df7_senior_pop)

# geometry to coordinates
df11_elementary_district['coordinates'] = df11_elementary_district['geometry'].apply(MULTIPOLYGON_to_coordinates)

# geometry to coordinates
df21_sidewalk['coordinates'] = df21_sidewalk['geometry'].apply(MULTILINESTRING_to_coordinates)
# df21_sidewalk = pd.DataFrame(df21_sidewalk)

# geometry to coordinates
df23_road['coordinates'] = df23_road['geometry'].apply(MULTILINESTRING_to_coordinates)
# df23_road = pd.DataFrame(df23_road)

# geometry to coordinates
df27_building['coordinates'] = df27_building['geometry'].apply(MULTIPOLYGON_to_coordinates)
# df27_building = pd.DataFrame(df27_building)

# geometry to coordinates
df28_grid_building['coordinates'] = df28_grid_building['geometry'].apply(MULTIPOLYGON_to_coordinates)
# df28_grid_building = pd.DataFrame(df28_grid_building)

# geometry to coordinates
df_osan_grid['coordinates'] = df_osan_grid['geometry'].apply(MULTIPOLYGON_to_coordinates)
# df_osan_grid = pd.DataFrame(df_osan_grid)

> 2. 시각화

In [ ]:
# layer = pdk.Layer(
#     'ScatterplotLayer',
#     df16_road_sign,
#     get_position=['lon', 'lat'],
#     get_radius=15,
#     get_fill_color=[180, 0, 200, 140],
#     pickable=True,
#     auto_highlight=True
#     )

# center=[127.0772,37.1498]
# view_state = pdk.ViewState( 
#     longitude=center[0], 
#     latitude=center[1], 
#     zoom=10
# ) 
# r2 = pdk.Deck(layers=[layer], initial_view_state=view_state,
#              map_style='mapbox://styles/mapbox/outdoors-v11',
#              mapbox_key = "pk.eyJ1IjoicmVib3JuMTk5OCIsImEiOiJja2oyZGppZ24wdHJ1MnRtaHU5dm92cnV0In0.8sNxBdHqt8JccQZB-oe3Cg"
#             )

# r2.to_html()

In [ ]:
# df23_road['N_road_width'] = df23_road['width'] / df23_road['width'].max()
# layer_1 = pdk.Layer(
#     'PathLayer',
#     df23_road,
#     get_path='coordinates',
#     get_width='width * 10',
#     get_color='[255, 120, 255* N_road_width]',
#     pickable=True,
#     auto_highlight=True
# )

# layer_2 = pdk.Layer(
#     'ScatterplotLayer',
#     df16_road_sign,
#     get_position=['lon', 'lat'],
#     get_radius=8,
#     get_fill_color=[180, 0, 200, 140],
#     pickable=True,
#     auto_highlight=True
#     )

# center=[127.0772,37.1498]
# view_state = pdk.ViewState(
#     longitude=center[0],
#     latitude=center[1],
#     zoom=13)

# r = pdk.Deck(layers=[layer_1, layer_2], initial_view_state=view_state,
#              map_style='mapbox://styles/mapbox/outdoors-v11',
#              mapbox_key = "pk.eyJ1IjoicmVib3JuMTk5OCIsImEiOiJja2oyZGppZ24wdHJ1MnRtaHU5dm92cnV0In0.8sNxBdHqt8JccQZB-oe3Cg"
#             )

# r.to_html()

In [ ]:
# df23_road['N_road_width'] = df23_road['width'] / df23_road['width'].max()
# layer_1 = pdk.Layer(
#     'PathLayer',
#     df23_road,
#     get_path='coordinates',
#     get_width='width * 10',
#     get_color='[255, 120, 255* N_road_width]',
#     pickable=True,
#     auto_highlight=True
# )

# layer_2 = pdk.Layer(
#     'ScatterplotLayer',
#     df16_road_sign,
#     get_position=['lon', 'lat'],
#     get_radius=8,
#     get_fill_color=[180, 0, 200, 140],
#     pickable=True,
#     auto_highlight=True
#     )

# layer_2 = pdk.Layer(
#     'ScatterplotLayer',
#     df10_school,
#     get_position=['lon', 'lat'],
#     get_radius=16,
#     get_fill_color=[180, 0, 200, 140],
#     pickable=True,
#     auto_highlight=True
#     )

# center=[127.0772,37.1498]
# view_state = pdk.ViewState(
#     longitude=center[0],
#     latitude=center[1],
#     zoom=13)

# r = pdk.Deck(layers=[layer_1, layer_2], initial_view_state=view_state,
#              map_style='mapbox://styles/mapbox/outdoors-v11',
#              mapbox_key = "pk.eyJ1IjoicmVib3JuMTk5OCIsImEiOiJja2oyZGppZ24wdHJ1MnRtaHU5dm92cnV0In0.8sNxBdHqt8JccQZB-oe3Cg"
#             )

# r.to_html()

In [ ]:
# df = df24_traffic
# df['N_total_traffic'] = df['all_traffic'] / df['all_traffic'].max()
# layer = pdk.Layer( 'PathLayer', 
#                   df, 
#                   get_path='link_id', 
#                   get_width='N_total_traffic*10', 
#                   get_color='[255, 255 * N_total_traffic, 120]', 
#                   pickable=True, auto_highlight=True 
#                  ) 

# center=[127.0772,37.1498]
# view_state = pdk.ViewState(
#     longitude=center[0],
#     latitude=center[1],
#     zoom=13)

# r = pdk.Deck(layers=[layer], initial_view_state=view_state,
#              map_style='mapbox://styles/mapbox/outdoors-v11',
#              mapbox_key = "pk.eyJ1IjoicmVib3JuMTk5OCIsImEiOiJja2oyZGppZ24wdHJ1MnRtaHU5dm92cnV0In0.8sNxBdHqt8JccQZB-oe3Cg"
#             )

# r.to_html()

In [ ]:
# layer = pdk.Layer('PathLayer', 
#                   df, 
#                   get_path='link_id', 
#                   get_width='N_total_traffic*10', 
#                   get_color='[255, 255 * N_total_traffic, 120]', 
#                   pickable=True, auto_highlight=True 
#                  ) 

# center=[127.0772,37.1498]
# view_state = pdk.ViewState(
#     longitude=center[0],
#     latitude=center[1],
#     zoom=13)

# r = pdk.Deck(layers=[layer], initial_view_state=view_state,
#              map_style='mapbox://styles/mapbox/outdoors-v11',
#              mapbox_key = "pk.eyJ1IjoicmVib3JuMTk5OCIsImEiOiJja2oyZGppZ24wdHJ1MnRtaHU5dm92cnV0In0.8sNxBdHqt8JccQZB-oe3Cg"
#             )

# r.to_html()

In [ ]:
# polygon_layer = pdk.Layer(
#     'PolygonLayer',
#     grid_info,
#     stroked=False,
#     # processes the data as a flat longitude-latitude pair
#     get_polygon="coordinates",
#     get_fill_color = '[180 * accident_cnt, 0, 200, 140]',
#     pickable=True,
#     auto_highlight=True
#     )

# layer = pdk.Layer(
#     'ScatterplotLayer',
#     df10_school,
#     get_position=['lon', 'lat'],
#     get_radius=15,
#     get_fill_color=[180, 0, 200, 140],
#     pickable=True,
#     auto_highlight=True
#     )

# center=[127.0772,37.1498]
# view_state = pdk.ViewState(
#     longitude=center[0],
#     latitude=center[1],
#     zoom=13)

# r = pdk.Deck(layers=[polygon_layer, layer], initial_view_state=view_state,
#              map_style='mapbox://styles/mapbox/outdoors-v11',
#              mapbox_key = "pk.eyJ1IjoicmVib3JuMTk5OCIsImEiOiJja2oyZGppZ24wdHJ1MnRtaHU5dm92cnV0In0.8sNxBdHqt8JccQZB-oe3Cg"
#             )

# r.to_html()

# 4. 격자 결합
> df1_parking: 격자별 불법주정차 단속지점 수 parking_val

> df4_pop: 격자별 인구수 pop_val

> df5_junior_pop: 격자별 아이 인구수 junior_val

> df8_floating_pop: 격자별 유동인구 수 floating_pop_val
    
> df10_school: 격자별 학교 수 school_val

> df13_kinder: 격자별 유치원 수 kinder_val

> df17_crosswalk: 오산시_횡단보도 crosswalk_val

> df18_speed_bump:오산시_과속방지턱 수 bump_val

> df20_cctv: 오산시_CCTV수 cctv_val

> df21_sidewalk: 오산시_인도 sidewalk_val

> df22_bus_stop: 격자별 버스 정류장 수 Bus_stop_val

> df28_grid_building: 격자별 빌딩 밀도 density_building_val
    
> df29_sports: 격자별 체육시설 수 sports_val

> df30_academy: 격자별 학원 수 academy_val


In [ ]:
# 인구 수와 유소년 인구 수 concat
grid_info=pd.concat([df4_pop,df5_junior_pop['junior_val']],axis=1)
grid_info

# 격자별 건물 수 concat 
grid_info=pd.concat([grid_info,df28_grid_building['grid_building_val']],axis=1)
grid_info

#grid_info와 df18_speed_bump를 합치기 위하여 변경
grid_info=gpd.GeoDataFrame(grid_info)
df18_speed_bump=location_to_point(df18_speed_bump)

grid_info=sjoin(grid_info,df18_speed_bump,'bump_val')

#유동인구를 함치기 위하여 변경
df8_floating_pop=location_to_point(df8_floating_pop)


temp=df8_floating_pop.groupby(['lon','lat'])['8_23'].mean()
temp=pd.DataFrame(temp)
temp=temp.reset_index()
# sjoin을 위하여 geodataframe으로 변경
temp=gpd.GeoDataFrame(temp)
temp=location_to_point(temp)
# grid_info와 sjoin
test=gpd.sjoin(grid_info,temp,op='contains')
test = test.groupby('gid')['8_23'].sum().to_frame().reset_index()
grid_info=pd.merge(grid_info,test,how='outer',on='gid')
grid_info=grid_info.rename(columns={'8_23':'floating_pop_val'})
#null값 처리
grid_info=grid_info.fillna(0)
grid_info

grid_info=grid_info[['gid','geometry','coordinates','pop_val','junior_val','grid_building_val','bump_val','floating_pop_val']]
grid_info

In [ ]:
school = df10_school.copy()
kinder = df13_kinder.copy()
sports = df29_sports.copy()
CCTV = df15_traffic_cctv.copy()
academy = df30_academy.copy()

# 창균
grid_info = sjoin(grid_info, df1_parking, 'pk_val')
grid_info = sjoin(grid_info, df22_bus_stop, 'bus_stop_val')
grid_info = sjoin(grid_info, df17_crosswalk, 'crosswalk_val')
grid_info = sjoin(grid_info, df21_sidewalk, 'sidewalk_val')

# 정민
grid_info = sjoin(grid_info, school, "school_val")
grid_info = sjoin(grid_info, kinder, 'kinder_val')
grid_info = sjoin(grid_info, sports, 'sports_val')
grid_info = sjoin(grid_info, academy, 'academy_val')
grid_info = sjoin(grid_info, CCTV, "traffic_cctv_val")

grid_info=sjoin(grid_info,df20_cctv,'cctv_val')
grid_info=sjoin(grid_info,df19_traffic_light,'traffic_light_val')

grid_info = grid_info.join(df2_grid_accident, how = 'left', lsuffix = '', rsuffix = '_r')
grid_info.drop(['gid_r', 'geometry_r'], axis = 1, inplace = True)

# 5. 버퍼 처리

1. 학교 버퍼

In [ ]:
grid_center = df_osan_grid['geometry'].centroid
# grid_center : 오산시 100m * 100m격자의 중앙점
dist_100m = Point(grid_center[0]).distance(Point(grid_center[1])) # 중앙점간의 거리는 100m
dist = dist_100m * 10
# dist => 300m



def point_to_buf(data_point, point_name,buf_name, n=5,): # parameter: data_point => 학교위치정보, 학원위치정보등등 point_name => school_name등, buf_name => buffer에 해당하는 해당 영역 1로 지정, n -> buf 거리지정
    global dist_100m
    dist = dist_100m * n
    
    data_buf = gpd.GeoDataFrame()
    data_buf['geometry'] = data_point.buffer(distance = dist, cap_style=1)
    data_buf = gpd.GeoDataFrame(data_point[point_name], geometry=data_buf.geometry)
    ret_df = gpd.overlay(df_osan_grid, data_buf, how='intersection')
    ret_df[buf_name] = 1
    
    return ret_df

In [ ]:
school_cores = point_to_buf(school,'school_name',"school_buffer_val" ,10)
school_cores=school_cores.groupby(['gid'])['school_buffer_val'].count()
school_cores=pd.DataFrame(school_cores).reset_index()
grid_info=pd.concat([grid_info,school_cores['school_buffer_val']],axis=1)
grid_info=grid_info.fillna(0)

2. 도로 링크 버퍼

In [ ]:
df23_road.columns
gdf_col = ['link_id', 'max_speed', 'road_name', 'road_no', 'road_rank',
       'link_type', 'road_type', 'up_lanes', 'dw_lanes', 'oneway', 'length',
       'width', 'car_lane', 'num_cross', 'barrier',]

gdf_road = gpd.GeoDataFrame(df23_road[gdf_col], geometry= df23_road.geometry)
gdf_road

In [ ]:
road_buf = gpd.GeoDataFrame()
road_buf['geometry'] = gdf_road.geometry.buffer(distance = (dist_100m/10), cap_style=3)
road_buf = gpd.GeoDataFrame(df23_road[gdf_col], geometry=road_buf.geometry)
road_buf = gpd.overlay(df_osan_grid, road_buf, how='intersection')

In [ ]:
road_buf.geometry.plot(figsize=(10,20))

In [ ]:

# layer_1 = pdk.Layer(
#     'PathLayer',
#     road_buf,
#     get_path='coordinates',
#     get_width='width * 10',
#     get_color='[255, 120, 255]',
#     pickable=True,
#     auto_highlight=True
# )
# grid_layer = pdk.Layer(
#     'PolygonLayer',
#     df_osan_grid,
#     stroked=False,
#     # processes the data as a flat longitude-latitude pair
#     get_polygon="coordinates",
#     get_fill_color = '[100, 0, 100, 140]',
#     pickable=True,
#     auto_highlight=True
# )

# center=[127.0772,37.1498]
# view_state = pdk.ViewState(
#     longitude=center[0],
#     latitude=center[1],
#     zoom=13
# )

# r = pdk.Deck(
#     layers=[layer_1, grid_layer,layer], initial_view_state=view_state,
#     map_style='mapbox://styles/mapbox/outdoors-v11',
#     mapbox_key = "pk.eyJ1IjoicmVib3JuMTk5OCIsImEiOiJja2oyZGppZ24wdHJ1MnRtaHU5dm92cnV0In0.8sNxBdHqt8JccQZB-oe3Cg"
# )

# r.to_html()

# 6. 격자 분석

In [ ]:
def normalization(data_set, column):
    m = data_set[column].min()
    M = data_set[column].max()
    data_set[column]=(data_set[column] - m)/(M - m)

In [ ]:
#정규화 과정
#한번만 돌려요
normalization_list=grid_info.columns[3:]
for i in normalization_list:
    normalization(grid_info,i)

정민 머신러닝 적용실습 

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor

grid_target = grid_info['accident_cnt']
y_target = grid_target
X_data = grid_info
X_data = X_data.drop(['gid', 'geometry', 'coordinates','accident_cnt'], axis=1, inplace=False)


rf = RandomForestRegressor(random_state = 0, n_estimators =1000)
neg_mse_scores = cross_val_score(rf, X_data, y_target, scoring="neg_mean_squared_error", cv=5)
rmse_scores = np.sqrt(-1*neg_mse_scores)
avg_rmse = np.mean(rmse_scores)

print(np.round(neg_mse_scores, 4))
print(np.round(rmse_scores, 2))
print(avg_rmse)
print(neg_mse_scores)

In [ ]:
def get_model_cv_prediction(model, X_data, y_target):
    neg_mse_scores = cross_val_score(model, X_data, y_target, scoring="neg_mean_squared_error", cv=5)
    rmse_scores = np.sqrt(-1*neg_mse_scores)
    avg_rmse = np.mean(rmse_scores)
    print("####", model.__class__.__name__, ' #####')
    print(' 5 교차 검증의 평균 RMSE : {0:.3f}'.format(avg_rmse))

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

dt_grid = DecisionTreeRegressor(random_state = 0, max_depth = 4)
gb_grid = GradientBoostingRegressor(random_state=0, n_estimators=1000)
xgb_grid = XGBRegressor(n_estimators=1000)
lgb_grid = LGBMRegressor(n_estimators=1000)
models = [dt_grid, rf, gb_grid,xgb_grid, lgb_grid]
for model in models:
    get_model_cv_prediction(model, X_data, y_target)

In [ ]:
import seaborn as sns
rf.fit(X_data, y_target)
feature_series = pd.Series(data = rf.feature_importances_, index=X_data.columns)
feature_series = feature_series.sort_values(ascending=False)
sns.barplot(x=feature_series, y=feature_series.index)

lgb_grid.fit(X_data, y_target)
feature_series_lgb = pd.Series(data=lgb_grid.feature_importances_, index=X_data.columns)
feature_series_lgb = feature_series_lgb.sort_values(ascending=False)
sns.barplot(x=feature_series_lgb, y=feature_series_lgb.index)

In [ ]:
rf_pred = rf.predict(X_data)
a = pd.Series(rf_pred)
# a = a.rename(columns = {0:'Pred'})
b = pd.Series(y_target)

# a['accident_cnt'] = b['accident_cnt']
a.plot()

In [ ]:
a = a.sort_values(ascending=False)
a.head(20)

In [ ]:
b.plot()
b = b.sort_values(ascending=False)
b.head(20)

In [ ]:
#회귀분석
corr_list=grid_info.columns[3:-1]
x=grid_info[corr_list]
y=grid_info['accident_cnt']
regression=linear_model.LinearRegression()
regression.fit(x,y)
print(regression.intercept_)
print(regression.coef_)

In [ ]:
from statsmodels.genmod.generalized_estimating_equations import GEE
from statsmodels.genmod.cov_struct import (Exchangeable,
    Independence,Autoregressive)
from statsmodels.genmod.families import Poisson

In [ ]:
fam = Poisson()
ind = Autoregressive()
model1 = GEE.from_formula("accident_cnt ~ pop_val + junior_val ", "gid", grid_info, cov_struct=ind, family=fam)
result1 = model1.fit()
print(result1.summary())

In [ ]:
import statsmodels.api as sm
from statsmodels.sandbox.regression.predstd import wls_prediction_std

In [ ]:
model = sm.OLS(y, x)
result=model.fit()
print(result.summary())

# Poisson

In [ ]:
from patsy import dmatrices
import statsmodels.api as sm
from scipy import stats

In [ ]:
#상관관계 heatmap 시각화
import seaborn as sns
corr=grid_info.corr(method='pearson')
plt.figure(figsize=(10,10))
sns.heatmap(data = corr, annot=True,fmt = '.2f', linewidths=.5, cmap='Blues')

In [ ]:
# expr = """accident_cnt ~ pop_val + junior_val + grid_building_val + bump_val + floating_pop_val + pk_val
#         + bus_stop_val + crosswalk_val + sidewalk_val + school_val + kinder_val
#         + sports_val + academy_val + cctv_val + traffic_light_val + school_buffer_val"""

expr = """accident_cnt ~ floating_pop_val + pk_val
        + bus_stop_val + crosswalk_val + sidewalk_val + school_val + kinder_val
        + sports_val + cctv_val + traffic_light_val + school_buffer_val"""

y, X = dmatrices(expr, grid_info, return_type='dataframe')

poisson_fit = sm.GLM(y, X, family=sm.families.Poisson()).fit()
poisson_predict = poisson_fit.predict()

a = pd.DataFrame(poisson_predict)
a = a.rename(columns = {0:'Pred'})
b = pd.DataFrame(grid_info['accident_cnt'])

a['accident_cnt'] = b['accident_cnt']
normalization(a, 'Pred')

In [ ]:
a['Pred'].plot()

In [ ]:
a['accident_cnt'].plot()

In [ ]:
# RMSE 너무 큼
a['residual'] = a['Pred'] - a['accident_cnt']
a['residual'] = a['residual'] ** (1/2)
a['residual'].sum()

In [ ]:
print(poisson_fit.summary())

In [ ]:
# 평균과 분산이 같지 않아 포아송 분포를 사용할 수 없다 가 맞지만 교수님은 poisson 쓰라고 하심.
print(a['Pred'].mean())
print(a['Pred'].var())

# Negative Binomial

In [ ]:
#상관관계 heatmap 시각화
import seaborn as sns
corr=grid_info.corr(method='pearson')
plt.figure(figsize=(20,10))
sns.heatmap(data = corr, annot=True,fmt = '.2f', linewidths=.5, cmap='Blues')

In [ ]:
# expr = """accident_cnt ~ pop_val + junior_val + grid_building_val + bump_val + floating_pop_val + pk_val
#         + bus_stop_val + crosswalk_val + sidewalk_val + school_val + kinder_val
#         + sports_val + academy_val + cctv_val + cctv_val + traffic_light_val + school_buffer_val"""

expr = """accident_cnt ~ pop_val + junior_val + grid_building_val + bump_val + floating_pop_val + pk_val
        + bus_stop_val + crosswalk_val + sidewalk_val + school_val + kinder_val
        + sports_val + academy_val + cctv_val + cctv_val + traffic_light_val + school_buffer_val"""

y, X = dmatrices(expr, grid_info, return_type='dataframe')

NB_fit = sm.GLM(y, X, family=sm.families.NegativeBinomial()).fit()

NB_predict = NB_fit.predict()

c = pd.DataFrame(NB_predict)
c = c.rename(columns = {0:'Pred'})
d = pd.DataFrame(grid_info['accident_cnt'])

c['accident_cnt'] = d['accident_cnt']
normalization(c, 'Pred')

print(NB_fit.summary())

In [ ]:
c['accident_cnt'].plot()

In [ ]:
c['Pred'].plot()

## 필요한 변수 추출

In [ ]:
# expr = """accident_cnt ~ pop_val + junior_val + grid_building_val + bump_val + floating_pop_val + pk_val
#         + bus_stop_val + crosswalk_val + sidewalk_val + school_val + kinder_val
#         + sports_val + academy_val + cctv_val + cctv_val + traffic_light_val + school_buffer_val"""

expr = """accident_cnt ~ junior_val + floating_pop_val + pk_val
        + bus_stop_val + crosswalk_val + sidewalk_val + school_val + kinder_val
        + sports_val + academy_val + cctv_val + traffic_cctv_val + traffic_light_val + school_buffer_val"""

y, X = dmatrices(expr, grid_info, return_type='dataframe')

NB_fit = sm.GLM(y, X, family=sm.families.NegativeBinomial()).fit()

NB_predict = NB_fit.predict()

e = pd.DataFrame(NB_predict)
e = e.rename(columns = {0:'Pred'})
f = pd.DataFrame(grid_info['accident_cnt'])

e['accident_cnt'] = f['accident_cnt']
normalization(e, 'Pred')

print(NB_fit.summary())

In [ ]:
e['Pred'].plot()

In [ ]:
e['accident_cnt'].plot()

In [ ]:
df2_grid_accident['accident_cnt'].plot()